In [ ]:
!pip install qubovert
!pip install qiskit-aqua
!pip install qiskit

In [68]:
import qiskit
qiskit.__version__

'0.19.1'

In [69]:
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance

from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

import math
import numpy as np
import pandas as pd
from sympy import *
import re
import time
import random
import itertools

##Benchmark Dataset generator

In [70]:
def dataset(i,y,Power,Numofagent):

    if (i == 1): #Agent Based uniform
        strx = bin(y)[2:]
        length = len(strx)
        Tempcoal = []
        for x in range(0, length):
            if (int(strx[x]) == 1):
                Tempcoal.append(length - x)


        sums = 0
        for j in Tempcoal :
            partial = np.random.uniform(0, 2 * Power.get(j))
            sums = sums + partial

        return sums




    if(i==2):#Agent based normal
        strx = bin(y)[2:]
        length = len(strx)
        Tempcoal = []
        for x in range(0, length):
            if (int(strx[x]) == 1):
                Tempcoal.append(length - x)


        sums = 0
        for j in Tempcoal:
            partial = np.random.normal(Power.get(j), 0.01)
            sums = sums + partial

        return sums



    if (i == 3): # Beta distribution
        return bin(y).count('1')* np.random.beta(.5, .5)

    if (i == 4): #odified normal distribution
        mu = 10 * bin(y).count('1')
        sigma = 0.01
        value = np.random.normal(mu, sigma)
        randval = np.random.uniform(0, 50)

        if randval <= 20:
            value = value + randval

        return value


    if (i == 5):#Modified uniform distribution

        value= np.random.uniform(0,10*bin(y).count('1'))
        randval = np.random.uniform(0, 50)

        if randval <= 20:
            value = value + randval

        return  value

    if (i == 6):#Normal distribution
        mu = 10 * bin(y).count('1')
        sigma = 0.01
        return np.random.normal(mu, sigma)

    if (i == 7):  # SVA BETA Distribution
        strx = bin(y)[2:]
        length = len(strx)
        Tempcoal = []
        for x in range(0, length):
            if (int(strx[x]) == 1):
                Tempcoal.append(length - x)
        x = len(Tempcoal)
        inputx = x * np.random.beta(0.5, 0.5)

        if (strx[len(strx) - 1] == str(1)):
            inputx = 200 + inputx
        return inputx

    if (i == 8):  # Weibull Distribution
        lengthofcoal = bin(y).count('1')
        value = np.random.weibull(lengthofcoal * lengthofcoal)
        return value

    if (i == 9):  # Rayleigh distribution
        lengthofcoal = bin(y).count('1')
        mu = 10 * lengthofcoal
        modevalue = np.sqrt(2 / np.pi) * mu
        value = np.random.rayleigh(modevalue)
        return value

    if (i == 10):  # Weighted random with chisquare
        lengthofcoal = bin(y).count('1')
        value = random.randint(1, lengthofcoal)
        inputx = lengthofcoal * np.random.chisquare(lengthofcoal)
        totalvalue = value + inputx
        return totalvalue

    if (i == 11):  # F distribution
        lengthofcoal = bin(y).count('1')

        dfden = lengthofcoal + 1
        dfnum = 1
        value = np.random.f(dfnum, dfden)
        return value
    if (i == 12):  # Laplace or double exponential
        lengthofcoal = bin(y).count('1')
        value = np.random.laplace(10 * lengthofcoal, 0.1)
        return value


def generate_problem_instance(distribution, NumAgent):
    distributions = ['Agent Based uniform', 'Agent based normal', 
                     'Beta distribution', 'odified normal distribution', 
                     'Modified uniform distribution', 'Normal distribution', 
                     'SVA BETA Distribution', 'Weibull Distribution', 
                     'Rayleigh distribution', 'Weighted random with chisquare', 
                     'F distribution', 'Laplace or double exponential']
    Datadistribution = distributions.index(distribution) + 1
    totalcoal = 2 ** NumAgent
    y = 1
    Agents = list(range(1, NumAgent + 1))
    RealAgent = Agents[0:NumAgent]
    Power = {}
    for i in RealAgent:
        Power[i] = np.random.uniform(0, 10)
    coalition_values = {}
    while (y < totalcoal):
        sums = float("{0:.3f}".format(dataset(Datadistribution,y,Power,NumAgent)))
        coalition_values[','.join([str(idx+1) for idx,bit in enumerate(bin(y)[2:][::-1]) if int(bit)])] = sums
        y = y + 1
    return coalition_values

In [71]:
def convert_to_BILP(coalition_values):
  '''

  '''
  x={}
  for i in range(len(coalition_values)):
    x[i] = symbols(f'x_{i}')
  n = list(coalition_values.keys())[-1].count(',')+1                              #get number of agents
  S=[]
  for agent in range(n):
    temp = []
    for coalition in coalition_values.keys():
      if str(agent+1) in coalition:
        temp.append(1)                                                            #'1' if agent is present in coalition
      else:
        temp.append(0)                                                            #'0' if agent is not present in coalition
    S.append(temp)
  b=[1]*n                                                                         # vector b is a unit vector in case of BILP of CSGP
  c = list(coalition_values.values())                                             # vector of all costs (coalition values)
  return (c,S,b)

In [72]:
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
def atoi(text):
    return int(text) if text.isdigit() else text


def get_QUBO_coeffs(c,S,b,P):
  '''

  '''
  x={}
  for i in range(len(c)):
    x[i] = symbols(f'x_{i}')
  final_eq = simplify(sum([c_value*x[i] for i,c_value in enumerate(c)])+P*sum([expand((sum([x[idx] for idx,element in enumerate(agent) if element])-1)**2) for agent in S])) #simplify numerical equation
  linear={}
  quadratic={}
  for term in final_eq.as_coeff_add()[1]:
    term = str(term)
    if '**' in term:                                                              #get the coefficient of the squared terms as linear coefficients (diagonal elements of the Q matrix in the QUBO problem)
      if term.count('*')==3:
        linear[term.split('*')[1]] = float(term.split('*')[0])
      else:
        if not term.startswith('-'):
          linear[term.split('**')[0]] = float(1)
        else:
          linear[term.split('**')[0][1:]] = float(-1)
    elif term.count('*')==1:                                                        #get the coefficient of the linear terms (diagonal elements of the Q matrix in the QUBO problem)
      linear[term.split('*')[1]] += float(term.split('*')[0])
    else:
      quadratic[(term.split('*')[1],term.split('*')[2])] = float(term.split('*')[0])  #get the coefficient of quadratic terms (upper diagonal elements of the Q matrix of the QUBO problem)  
  return linear,quadratic

In [135]:
def solve_QUBO(linear, quadratic, algo='qaoa', p=1):
  '''
  
  '''
  keys = list(linear.keys())
  keys.sort(key=natural_keys)
  linear = [linear[key] for key in keys]
  qubo = QuadraticProgram()
  for i in range(len(linear)):
    qubo.binary_var(f'x_{i}')                                                             #initialize the binary variables
  qubo.maximize(linear=linear, quadratic=quadratic)                                       #initialize the QUBO maximization problem instance
  op, offset = qubo.to_ising()
  qp=QuadraticProgram()
  qp.from_ising(op, offset, linear=True)
  aqua_globals.random_seed = 123
  quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),       #qasm simulator
                                     seed_simulator=aqua_globals.random_seed,
                                     seed_transpiler=aqua_globals.random_seed)
  if algo == 'qaoa':
    qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
    qaoa = MinimumEigenOptimizer(qaoa_mes)    # using QAOA
    result = qaoa.solve(qubo)
    #print('result.variables_dict',result.variables_dict)
    #print('result.raw_results',result.raw_results)
    #print(qaoa_mes.get_probabilities_for_counts(qaoa_mes.get_optimal_vector()))
  else:
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
    result = exact.solve(qubo)
  return result

In [136]:
#convert solution binary string to coalition structure and also output the coalition
def decode(solution,coalition_values):
  '''

  '''
  output = []
  for index,element in enumerate(solution):
    if int(element)!=0:
      output.append(set(list(coalition_values)[index].split(',')))
  return output

In [137]:
def get_linear_quads(distribution,n):
  coalition_values = generate_problem_instance(distribution, n)
  c,S,b = convert_to_BILP(coalition_values)
  qubo_penalty =-50
  linear,quadratic = get_QUBO_coeffs(c,S,b,qubo_penalty)
  return coalition_values,linear,quadratic

##Classical Solver

In [138]:
#Classical BILP solver

def constraint(x,S):
  '''
  
  '''
  for i in range(len(S)):
    temp = 0
    for j in range(len(x)):
      temp+= (S[i][j]*int(x[j]))
    if temp!=1:
      return False
  return True


def solve_BILP_classical(coalition_values):
  '''
  
  '''
  S=[]
  for i in range(math.ceil(math.sqrt(len(coalition_values)))):                    #get number of agents
    S.append([])
    for j,value in coalition_values.items():
      if str(i+1) in j:
        S[i].append(1)
      else:
        S[i].append(0)
  optimal_cost = 0
  for b in range(2**len(coalition_values)):
    x = [int(t) for t in reversed(list(bin(b)[2:].zfill(len(coalition_values))))]
    x = ''.join(str(e) for e in x)
    if constraint(x,S):
      cost = 0
      for j in range(len(x)):
        cost+=coalition_values[list(coalition_values.keys())[j]]*int(x[j])
      if optimal_cost<cost:
        optimal_cost = cost
        optimal_x = x
  return optimal_x

##Execute Functions and Generate Table

In [150]:
table_headers = ['Distribution', 'No. of Agents','Solution Function Value','Solution Binary String','QAOA Solution','True Solution (Classical)','QAOA Result']
table_contents = []


distributions = ['Agent Based uniform', 'Agent based normal', 
                     'Beta distribution', 'odified normal distribution', 
                     'Modified uniform distribution', 'Normal distribution', 
                     'SVA BETA Distribution', 'Weibull Distribution', 
                     'Rayleigh distribution', 'Weighted random with chisquare', 
                     'F distribution', 'Laplace or double exponential']
#distributions = ['Agent Based uniform']
n_agents = [2,3]
#n_agents = [2]

for distribution in distributions:
  for n in n_agents:
    start_time = time.time()
    print(f'Executing "{distribution}" distribution for {n} agents',end='')
    row = []
    coalition_values, linear, quadratic = get_linear_quads(distribution,n)
    p = 3                                            # number of QAOA layers
    solution = solve_QUBO(linear,quadratic,'qaoa',p)
    optimal_coalition_structure = decode(solution.x,coalition_values)

    row.append(distribution)
    row.append(n)
    row.append(solution.fval)
    row.append(list(map(int,solution.x)))
    qaoa_solution = decode(solution.x,coalition_values)
    row.append(qaoa_solution)
    print(f', Time taken = {time.time()-start_time}')
    classical_solution = decode(list(solve_BILP_classical(coalition_values)),coalition_values)
    row.append(classical_solution)     #Calculating classical solution
    row.append(classical_solution == qaoa_solution)
    table_contents.append(row)

Executing "Agent Based uniform" distribution for 2 agents, Time taken = 0.4259798526763916
Executing "Agent Based uniform" distribution for 3 agents, Time taken = 2.325193166732788
Executing "Agent based normal" distribution for 2 agents, Time taken = 0.43282389640808105
Executing "Agent based normal" distribution for 3 agents, Time taken = 2.247495651245117
Executing "Beta distribution" distribution for 2 agents, Time taken = 0.4025387763977051
Executing "Beta distribution" distribution for 3 agents, Time taken = 2.2156598567962646
Executing "odified normal distribution" distribution for 2 agents, Time taken = 0.40404224395751953
Executing "odified normal distribution" distribution for 3 agents, Time taken = 2.1441211700439453
Executing "Modified uniform distribution" distribution for 2 agents, Time taken = 0.4105982780456543
Executing "Modified uniform distribution" distribution for 3 agents, Time taken = 2.1586811542510986
Executing "Normal distribution" distribution for 2 agents, T

In [151]:
#view output table
def highlight_false(s, column):
    is_false = pd.Series(data=False, index=s.index)
    is_false[column] = s.loc[column] == False
    return ['background-color: red' if is_false.any() else '' for v in is_false]






df = pd.DataFrame(table_contents, columns=table_headers)


df.style.apply(highlight_false, column='QAOA Result', axis=1)

,Distribution,No. of Agents,Solution Function Value,Solution Binary String,QAOA Solution,True Solution (Classical),QAOA Result
0,Agent Based uniform,2,107.490000,"[0, 0, 1]","[{'2', '1'}]","[{'2', '1'}]",True
1,Agent Based uniform,3,168.890000,"[0, 1, 0, 0, 1, 0, 0]","[{'2'}, {'1', '3'}]","[{'2'}, {'1', '3'}]",True
2,Agent based normal,2,110.722000,"[1, 1, 0]","[{'1'}, {'2'}]","[{'1'}, {'2'}]",True
3,Agent based normal,3,166.356000,"[1, 1, 0, 1, 0, 0, 0]","[{'1'}, {'2'}, {'3'}]","[{'1'}, {'2'}, {'3'}]",True
4,Beta distribution,2,101.961000,"[1, 1, 0]","[{'1'}, {'2'}]","[{'1'}, {'2'}]",True
5,Beta distribution,3,152.737000,"[0, 0, 0, 0, 0, 0, 1]","[{'2', '1', '3'}]","[{'2', '1', '3'}]",True
6,odified normal distribution,2,128.747000,"[0, 0, 1]","[{'2', '1'}]","[{'2', '1'}]",True
7,odified normal distribution,3,196.793000,"[0, 0, 0, 0, 0, 0, 1]","[{'2', '1', '3'}]","[{'2', '1', '3'}]",True
8,Modified uniform distribution,2,125.393000,"[1, 1, 0]","[{'1'}, {'2'}]","[{'1'}, {'2'}]",True
9,Modified uniform distribution,3,200.549000,"[0, 1, 0, 0, 1, 0, 0]","[{'2'}, {'1', '3'}]","[{'2'}, {'1', '3'}]",True


In [6]:
#Classical BILP solver     2,3,4,5(takes a lot of time)

#Classical QUBO solver     2,3,4,5(resource overflow)gith

#QAOA QUBO solver          2,3,4(15 qubits)(resource overflow),5(31 qubits)(resource overflow)